In [40]:
import graspy 

import pandas as pd 
import matplotlib as plt 
import numpy as np 


x = np.array([[(0,0),(0,1),(0,2),(0,3)],[(1,0),(1,1),(1,2),(1,3)],[(2,0),(2,1),(2,2),(2,3)],[(3,0),(3,1),(3,2),(3,3)]])

In [53]:
n = 4 
k = np.array([[(0,0),(0,1),(0,2),(0,3),(1,0),(1,1),(1,2),(1,3)],[(2,0),(2,1),(2,2),(2,3),(3,0),(3,1),(3,2),(3,3)]])

print(k)
#sum (each row of k is n^2)\
i = 0

k.shape[0]

len(k[1])

[[[0 0]
  [0 1]
  [0 2]
  [0 3]
  [1 0]
  [1 1]
  [1 2]
  [1 3]]

 [[2 0]
  [2 1]
  [2 2]
  [2 3]
  [3 0]
  [3 1]
  [3 2]
  [3 3]]]


AttributeError: 'int' object has no attribute 'type'

In [54]:
k.size/2 != n**2/2

True

In [42]:
x = np.array([[(0,0),(0,1),(0,2),(0,3)],[(1,0),(1,1),(1,2),(1,3)],[(2,0),(2,1),(2,2),(2,3)],[(3,0),(3,1),(3,2),(3,3)]])

x.shape

(4, 4, 2)

In [12]:
n_flat = np.ravel(n)
print(n_flat)
n_unflat = np.unravel_index(n_flat[4],shape = (1,2))
print(n_unflat)
np.unravel_index(6,(3,4))


(1, 2)

In [66]:
def siem(n, p,edge_comm, directed=False, loops=False, wt=None, wtargs=None):
    """
    Samples a graph from the structured independent edge model (SIEM) 

    SIEM produces a graph with specified communities, in which each community can
    have different sizes and edge probabilities. 

    Read more in the :ref:`tutorials <simulations_tutorials>`

    Parameters
    ----------
    n: int
        Number of vertices

    p: list of int of length K (k_communities)
        Probability of an edge existing within the corresponding communities, where p[i] indicates 
        the probability of an edge existing in the edge_comm[i]
    
    edge_comm: list of K tuples (k_communities)
        tuple is the indices for the edge within the kth community.

    directed: boolean, optional (default=False)
        If False, output adjacency matrix will be symmetric. Otherwise, output adjacency
        matrix will be asymmetric.

    loops: boolean, optional (default=False)
        If False, no edges will be sampled in the diagonal. Otherwise, edges
        are sampled in the diagonal.

    wt: object or list of K objects (k_communities)
        if Wt is an object, a weight function to use globally over
        the siem for assigning weights. If Wt is a list, a weight function for each of
        the edge communities to use for connection strengths Wt[i] corresponds to the weight function
        for edge community i. Default of None results in a binary graph

    wtargs: dictionary or array-like, shape (k_communities)
        if Wt is an object, Wtargs corresponds to the trailing arguments
        to pass to the weight function. If Wt is an array-like, Wtargs[i, j] 
        corresponds to trailing arguments to pass to Wt[i, j].

    return_labels: boolean, optional (default = False)
        IF True, returns the edge-communities as well

    References
    ----------

    Returns
    -------
    A: ndarray, shape (sum(n), sum(n))
        Sampled adjacency matrix
        
    T: returns the edge-communities if return_labels == True

    Examples
    --------
    >>> np.random.seed(1)
    >>> n = [3, 3]
    >>> p = [[0.5, 0.1], [0.1, 0.5]]

    To sample a binary 2-block SBM graph:

    >>> sbm(n, p)
    array([[0., 0., 1., 0., 0., 0.],
           [0., 0., 1., 0., 0., 1.],
           [1., 1., 0., 0., 0., 0.],
           [0., 0., 0., 0., 1., 0.],
           [0., 0., 0., 1., 0., 0.],
           [0., 1., 0., 0., 0., 0.]])

    To sample a weighted 2-block SBM graph with Poisson(2) distribution:

    >>> wt = np.random.poisson
    >>> wtargs = dict(lam=2)
    >>> sbm(n, p, wt=wt, wtargs=wtargs)
    array([[0., 4., 0., 1., 0., 0.],
           [4., 0., 0., 0., 0., 2.],
           [0., 0., 0., 0., 0., 0.],
           [1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 2., 0., 0., 0., 0.]])
    """
    # Check n
    if not isinstance(n, (int)):
        msg = "n must be a int, not {}.".format(type(n))
        raise TypeError(msg)

    # Check p
    if not isinstance(p, (list, np.ndarray)):
        msg = "p must be a list or np.array, not {}.".format(type(p))
        raise TypeError(msg)
    else:
        p = np.array(p)
        if not np.issubdtype(p.dtype, np.number):
            msg = "There are non-numeric elements in p"
            raise ValueError(msg)
        elif np.any(p < 0) or np.any(p > 1):
            msg = "Values in p must be in between 0 and 1."
            raise ValueError(msg)

    # Check edge_comm 
    if not isinstance(edge_comm, (list,np.ndarray)):
        msg = "edge_comm must be a list of tuples of length K or a 2-d array of K tuples"
    else: 
        edge_comm = np.array(edge_comm)
        #generate temporary adjacency matrix to check upper triangular?
        if (directed == True) and (loops == True):
            if edge_comm.size/2 != n**2 : 
                msg = "Edge Communities and Number of Vertices Do Not Agree!"
                raise ValueError(msg)
        elif (directed == True) and (loops == False): 
            if edge_comm.size/2 != n*(n-1)/2 : 
                msg = "Edge Communities and Number of Vertices Do Not Agree!"
                raise ValueError(msg)            
        elif (directed == False) and (loops == True):
            #check symmetry ?
            if edge_comm.size/2 != n*(n-1) : 
                msg = "Edge Communities and Number of Vertices Do Not Agree!"
                raise ValueError(msg)            
        elif (directed == False) and (loops == False): 
            #check symmetry ?
            if edge_comm.size/2 != n**2/2 : 
                msg = "Edge Communities and Number of Vertices Do Not Agree!"
                raise ValueError(msg)            
            
    # Check wt and wtargs
    if (wt is not None) and (wtargs is not None): 
        if not callable(wt):
            # if not object, check dimensions
            if len(wt) != (edge_comm.shape[0]):
                msg = "wt must have size k, not {}".format(wt.shape)
                raise ValueError(msg)
            if len(wtargs) != (edge_comm.shape[0]):
                msg = "wtargs must have size k , not {}".format(wtargs.shape)
                raise ValueError(msg)
            # check if each element is a function
            for element in wt.ravel():
                if not callable(element):
                    msg = "{} is not a callable function.".format(element)
                    raise TypeError(msg)
        else:
            #extend the function to size of k 
            wt = np.full(edge_comm.shape[0], wt, dtype=object)
            wtargs = np.full(edge_comm.shape[0], wtargs, dtype=object)

    # Check directed
    if not directed:
        if np.any(p != p.T):
            raise ValueError("Specified undirected, but P is directed.")
        if np.any(wt != wt.T):
            raise ValueError("Specified undirected, but Wt is directed.")
        if np.any(wtargs != wtargs.T):
            raise ValueError("Specified undirected, but Wtargs is directed.")

    K = edge_comm.shape[0]  # the number of communities
    counter = 0
#     # get a list of community indices
#     cmties = []
#     for i in range(0, K):
#         cmties.append(range(counter, counter + n[i]))
#         counter += n[i]

    # End Checks, begin simulation
    A = np.zeros((n,n))
    
    for i in range(0, K):
        #sample edges randomly 
        A[i,:] = np.random.binomial(len(edge_comm[i]),p[i])
        
        #adjust adjacency matrix with any weight args. 
        if (wt is not None) and (wtargs is not None): 
            A[i,:] = A[i,:]*wt[i](**wtargs[i])
        
    if not directed:
        A = A + A.T - diag(A)
   
#     if not loops:
#         A = A - np.diag(np.diag(A))
#     if not directed:
#         A = symmetrize(A, method="triu")
    return A


In [67]:
siem(n = 4, p = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5] , edge_comm = k, directed = True, loops = True)

array([[6., 6., 6., 6.],
       [2., 2., 2., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])